In [1]:
import os
from transformers import AutoModelForCausalLM, AutoTokenizer

In [40]:
base_path = '/mnt/sxtn/chem/ChemLactica/checkpoints/facebook/'
model_size = 'galactica-125m'
run_hash = '2ecf54c441824491a3baebbb'
checkpoint_no = 'checkpoint-2048'

In [41]:
run_hash = 'dff5ace8f2ac45cc9681ab6a'
checkpoint_no = 'checkpoint-194560'
checkpoint_path = os.path.join(base_path, model_size, run_hash, checkpoint_no)

In [42]:
tokenizer = AutoTokenizer.from_pretrained('facebook/galactica-125m')
model = AutoModelForCausalLM.from_pretrained(checkpoint_path)

In [96]:
prompt = """[QED 0.84][FRACTIONCSP3 0.38][RINGCOUNT 2][START_SMILES]"""
#prompt = """[START_SMILES] C1=NC(=C(N1C2C(C(C(O2)COP(=O)(O)O)O)O)N)C(=O)N [END_SMILES] ["""
prompt = """[SAS 4.32] [START_SMILES]"""
# prompt = """[NumHAcceptors"""

In [97]:
inputs = tokenizer(prompt, return_tensors="pt").input_ids

In [98]:
outputs = model.generate(inputs, max_new_tokens=300, do_sample=True, top_k=1, top_p=0.95)
tokenizer.batch_decode(outputs, skip_special_tokens=False)

['[SAS 4.32] [START_SMILES] CC1=C(C(=C(C(=C1C)C)C2=C(C(=C(C(=C2C)C)C3=C(C(=C(C(=C3C)C)C)C)C)C)C)C)C [END_SMILES][NUMAROMATICHETEROCYCLES 0][NUMALIPHATICCARBOCacetMILAR 0.56 CC1=CC(=C(C=C1)C)C(=C)C2=CC=CC=C2C][SIMILAR 0.56 CC1=CC(=C(C=C1)C)C(=C)C2=CC=CC=C2C][SIMILAR 0.56 CC1=CC(=C(C=C1)C)C(=C)C2=CC=CC=C2C][SIMILAR 0.56 CC1=CC(=C(C=C1)C)C(=C)C2=CC=CC=C2C][SIMILAR 0.56 CC1=CC(=C(C=C1)C)C(=C)C2=CC=CC=C2C][']

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 21.7 MB/s eta 0:00:0000:0100:01


In [100]:
import rdkit.Chem as Chem
import sys
from rdkit.Chem import RDConfig, MACCSkeys
import numpy as np 
def tanimoto_similarity(smiles_1, smiles_2):
    fgp1 = np.array(MACCSkeys.GenMACCSKeys(Chem.MolFromSmiles(smiles_1)))
    fgp2 = np.array(MACCSkeys.GenMACCSKeys(Chem.MolFromSmiles(smiles_2)))

    both = np.sum(fgp1 & fgp2)

    return both / (np.sum(fgp1) + np.sum(fgp2) - both)
sys.path.append(os.path.join(RDConfig.RDContribDir, 'SA_Score'))
import sascorer
mol_source = Chem.MolFromSmiles("CC1=C(C(=C(C(=C1C)C)C2=C(C(=C(C(=C2C)C)C3=C(C(=C(C(=C3C)C)C)C)C)C)C)C)C")
mol_second = Chem.MolFromSmiles("CC1=CC(=C(C=C1)C)C(=C)C2=CC=CC=C2C")
source_sas = sascorer.calculateScore(mol_source)
print(tanimoto_similarity("CC1=C(C(=C(C(=C1C)C)C2=C(C(=C(C(=C2C)C)C3=C(C(=C(C(=C3C)C)C)C)C)C)C)C)C","CC1=CC(=C(C=C1)C)C(=C)C2=CC=CC=C2C"))

0.6666666666666666
